In [8]:
from math import sqrt
import numpy as np

In [4]:
L = 2         # m
A = 0.02      # m²
E = 200e9     # Pa | módulo de elasticidade
P = 50e3      # N

In [63]:
nn = 3
N = np.array([[0, L/2, L], [0, 0, 0]])

In [52]:
# configuração
N_ELEMENTS = 3
N_NODES = 3
MODULUS_OF_ELASTICITY = E # Módulo de elasticidade
AREA = A # Area

In [147]:
class Node():
    def __init__(self, x, y, value):
        self.value = value
        self.x = x
        self.dx = None
        self.y = y
        self.dy = None
        
    def __repr__(self):
        return f"#{self.value} ({self.x}, {self.y})"
    
    def dist(self, node):
        return sqrt((self.x - node.x)**2 + (self.y - node.y)**2)

class Element(): # barra
    def __init__(self, node_1:Node, node_2:Node, E=MODULUS_OF_ELASTICITY, A=AREA):
        self.node_1 = node_1
        self.node_2 = node_2
        self.length = node_1.dist(node_2)
        self.S = E*A/self.length

    @property
    def connectivity(self):
        connectivity_list = [0]*N_NODES
        connectivity_list[self.node_1.value] = -1
        connectivity_list[self.node_2.value] = 1
        connectivity_list = np.array( [ connectivity_list ] )
        return np.matmul(connectivity_list.T, connectivity_list)

    @property
    def stiffness(self):
        return self.S*self.connectivity
    
    def __repr__(self):
        return f"{self.node_1} -> {self.node_2}"

In [148]:
nodes = []
for i, (x, y) in enumerate(N.T):
    nodes.append(Node(x=N[0, i], y=N[1,i], value=i))

In [149]:
nodes

[#0 (0.0, 0.0), #1 (1.0, 0.0), #2 (2.0, 0.0)]

In [152]:
elementos = [
    Element(nodes[0], nodes[1]),
    Element(nodes[1], nodes[2])
]

In [153]:
print(elementos[0])
print(elementos[1])

#0 (0.0, 0.0) -> #1 (1.0, 0.0)
#1 (1.0, 0.0) -> #2 (2.0, 0.0)


In [154]:
elemento = elementos[1]

In [155]:
elemento.connectivity

array([[ 0,  0,  0],
       [ 0,  1, -1],
       [ 0, -1,  1]])

In [136]:
elemento.stiffness

array([[ 0.e+00,  0.e+00,  0.e+00],
       [ 0.e+00,  4.e+09, -4.e+09],
       [ 0.e+00, -4.e+09,  4.e+09]])

In [157]:
rigidez_total = sum([ el.stiffness for el in elementos ])
rigidez_total

array([[ 4.e+09, -4.e+09,  0.e+00],
       [-4.e+09,  8.e+09, -4.e+09],
       [ 0.e+00, -4.e+09,  4.e+09]])